Install the required libraries:

In [1]:
!pip install pandas networkx torch torch-geometric


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


2. Load and preprocess the data:

In [2]:
import pandas as pd

# Load the datasets
books = pd.read_csv('data/Books.csv')
ratings = pd.read_csv('data/Ratings.csv')
users = pd.read_csv('data/Users.csv')

# Preprocess the data (e.g., encode categorical variables, normalize ratings, etc.)
books['ISBN'] = books['ISBN'].astype('category').cat.codes
users['User-ID'] = users['User-ID'].astype('category').cat.codes
ratings['ISBN'] = ratings['ISBN'].astype('category').cat.codes
ratings['User-ID'] = ratings['User-ID'].astype('category').cat.codes

/var/folders/jm/l7z1pz590c340yqzvk57pbw00000gn/T/ipykernel_12148/136446664.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('data/Books.csv')


Create the graph:

In [3]:
import networkx as nx

# Create a bipartite graph
G = nx.Graph()

# Add nodes for users and books
G.add_nodes_from(users['User-ID'], bipartite=0)
G.add_nodes_from(books['ISBN'], bipartite=1)

# Add edges based on ratings
edges = list(zip(ratings['User-ID'], ratings['ISBN']))
G.add_edges_from(edges)

Define the GNN model:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GNNModel(nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

: 

Train the model:

In [ ]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

# Convert the NetworkX graph to a PyTorch Geometric data object
edge_index = torch.tensor(list(G.edges)).t().contiguous()
data = Data(x=torch.eye(G.number_of_nodes()), edge_index=edge_index)

# Create a DataLoader
loader = DataLoader([data], batch_size=1, shuffle=True)

# Initialize the model, loss function, and optimizer
model = GNNModel(num_features=G.number_of_nodes(), hidden_dim=16, num_classes=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

print(model)


In [ ]:

# Train the model
model.train()
for epoch in range(100):
    for batch in loader:
        optimizer.zero_grad()
        out = model(batch)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Make recommendations:

In [ ]:
model.eval()
with torch.no_grad():
    out = model(data)
    # Get the top N recommendations for a user
    User-ID = 0
    user_node = users[users['User-ID'] == User-ID].index[0]
    recommendations = out[user_node].argsort(descending=True)[:10]
    recommended_books = books.iloc[recommendations]
    print(recommended_books)